In [1]:
import sys
# need to specify path to remo in notebook
local_path_to_repo =  '/home/andrea/Desktop/Projects/repo/remo-python'

sys.path.insert(0, local_path_to_repo)


%load_ext autoreload
%autoreload 2
import remo

ImportError: cannot import name 'Annotation'

# Uploading Annotations

In this tutorial, we explore some different options to upload annotations in Remo. We will:
- introduce the concept of Annotation set
- add annotations using a file format supported by Remo
- add annotations directly from code
- add annotations using custom files formats

We start off by creating a dataset and populating it with some images

In [2]:
%load_ext autoreload
%autoreload 2
import remo
import os
import pandas as pd

urls = ['https://remo-scripts.s3-eu-west-1.amazonaws.com/open_images_sample_dataset.zip']
my_dataset = remo.create_dataset(name = 'D1', urls = urls)

Launching Remo server ...
Wait a bit...  5

    (\(\ 
    (>':') Remo server is running: {'app': 'remo', 'version': '0.3.10-53-g3012aa79'}
                    


## Annotation set

Annotation sets represent a collection of annotations for all the images within a Dataset.
An annotation set is characterized by a task (such as 'Object Detection') and a list of classes.

The advantage of grouping annotations in an Annotation Set is that it allows for high-level group operations on all the annotations, such:
- grouping classes together
- deleting objects of specific classes
- comparing of different annotations (such as ground truth vs prediction, or annotations coming from different annotators)


**Let's first create an empty annotation set with a predetermined list of classes**

In [28]:
my_classes = ['Airplane', 'Clothing', 'Dog', 'Fashion accessory', 'Food', 'Footwear', 'Fruit', 'Human arm', 
         'Human body', 'Human hand', 'Human leg', 'Mammal', 'Man', 'Person', 'Salad', 'Sports equipment', 'Trousers', 'Woman']

annotation_set = my_dataset.create_annotation_set(annotation_task = 'Object detection',
                                          name = 'Objects',
                                          classes = my_classes)

We can easily retrieve different annotation sets of a dataset

In [29]:
my_dataset.annotation_sets()

[Annotation set 17 - 'Objects', task: Object detection, #classes: 18]

## Add annotations from supported CSV file format

**Let's add some annotations from a CSV file**

In this example, annotations are stored in a CSV file in a format already supported by Remo. Class labels were encoded using [GoogleKnowledgeGraph](https://developers.google.com/knowledge-graph).

In [4]:
annotation_files=[os.getcwd() + '/assets/open_sample.csv']

df = pd.read_csv(annotation_files[0])
df.columns

Index(['ImageID', 'Source', 'LabelName', 'Confidence', 'XMin', 'XMax', 'YMin',
       'YMax', 'IsOccluded', 'IsTruncated', 'IsGroupOf', 'IsDepiction',
       'IsInside'],
      dtype='object')

When adding the file to the annotation set, remo automatically: 
- detects the class encoding and translates it into the corresponding labels
- only adds annotations for classes that are part of the existing annotation set

In [31]:
my_dataset.add_data(local_files=annotation_files, annotation_task = 'Object detection')

{'files_link_result': {'files uploaded': 0, 'annotations': 9, 'errors': []}}

In [32]:
my_dataset.get_annotation_statistics()

[{'AnnotationSet ID': 17,
  'AnnotationSet name': 'Objects',
  'n_images': 9,
  'n_classes': 18,
  'n_objects': 84,
  'top_3_classes': [{'name': 'Fruit', 'count': 27},
   {'name': 'Sports equipment', 'count': 12},
   {'name': 'Mammal', 'count': 7}],
  'creation_date': None,
  'last_modified_date': '2020-03-08T16:35:05.643905Z'}]

In [33]:
my_dataset.view()

Open http://localhost:8123/datasets/26


![dataset_added_annotation.jpeg](assets/dataset_added_annotation.jpeg) 

## Add annotations from code

**Let's see how we can add annotations to a specific image from code**

This can be useful for instance to add model predictions as annotations or to tag specific images.

First, let's retrieve one image

In [37]:
images = my_dataset.images()
my_image = images[1]
print(my_image)
print('Resoultion: ', my_image.width, 'x', my_image.height)

Image: 936 - 000a1249af2bc5f0.jpg
Resoultion:  1024 x 678


**Now we can add annotations using the Annotation class**

We can add an annotation object to either the annotation set or directly to an image

In [38]:
annotations = []

annotation = remo.Annotation()
annotation.classes='Human hand'
annotation.img_filename = '000a1249af2bc5f0.jpg'
annotation.bbox=[227, 284, 678, 674]
annotations.append(annotations)

annotation = remo.Annotation()
annotation.classes='Human hand'
annotation.bbox=[311, 193, 607, 526]
annotation.img_filename = '000a1249af2bc5f0.jpg'
annotations.append(annotations)

ds.add_annotations(annotations)

In [39]:
annotation = remo.Annotation()
annotation.add_item(classes=['Fashion accessory'], bbox=[496, 322, 544,370])

my_image.add_annotation(annotation)

In [40]:
my_image.view()

Open http://localhost:8123/image/936?dataset_id=26


![added_annotation.jpeg](assets/added_annotation.jpeg)

## TODO: Add annotations from custom file